- https://python.langchain.com/docs/modules/agents/how_to/add_memory_openai_functions
- https://python.langchain.com/docs/modules/agents/how_to/agent_vectorstore

- (SYSTEM MESSAGE TO AGENT) https://github.com/hwchase17/langchain/pull/6297 

### Tools
- https://python.langchain.com/docs/modules/agents/tools/how_to/multi_input_tool
- https://python.langchain.com/docs/modules/agents/tools/how_to/custom_tools
- https://python.langchain.com/docs/modules/agents/tools/integrations/google_search

In [1]:
%cd ..

D:\Proyectos DL\PyTorch-Tutorials\Open AI api chatbot


In [2]:
from dotenv import load_dotenv, find_dotenv
import os
import openai

In [3]:
load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [9]:
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper,
)
from langchain.prompts import SystemMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [19]:
### document loaders
file = 'docs/products.csv'
loader = CSVLoader(file_path=file)

products_docs = loader.load()
embeddings = OpenAIEmbeddings()

products_db = DocArrayInMemorySearch.from_documents(
    products_docs, 
    embeddings,
    collection_name="products_descriptions",
)


products_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=products_db.as_retriever()
)
### tools

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="Products_QA",
        func=products_qa.run,
        description="useful for when you need to answer questions about the products that the company offers. information provided in this document is item name, item description and unit price.",
    ),
]

In [20]:
system_template = """
You are a helpful AI assistant for MineSafe Solutions, a trusted provider of mining safety products. Your role is to assist potential buyers by answering their queries and providing information about our wide range of safety products. With our innovative solutions, we prioritize the highest level of safety and protection in the mining industry. Whether they're looking for personal protective equipment, safety devices, or advanced monitoring systems, you are here to guide them. Feel free to ask any questions or let them know how you can help improve safety at their mining operations.
"""

In [21]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "system_message": SystemMessage(content=system_template),
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
memory2 = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [22]:
def get_answer_agent(question, memory, verbose=True):
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=verbose,
        agent_kwargs=agent_kwargs,
        memory=memory,
    )
    
    response = agent.run(question)
    
    return response

In [29]:
query = 'what is your cheapest products'

In [30]:
get_answer_agent(query, 
                 memory)



> Entering new AgentExecutor chain...

Invoking: `Products_QA` with `cheapest`


The cheapest item is the High-Visibility Vests, which are priced at $15.Our cheapest product is the High-Visibility Vests, which are priced at $15. These vests are essential for ensuring the visibility and safety of workers in mining operations. They are made of high-quality materials and meet industry safety standards. If you would like more information about the High-Visibility Vests or any other products, please let me know!

> Finished chain.


'Our cheapest product is the High-Visibility Vests, which are priced at $15. These vests are essential for ensuring the visibility and safety of workers in mining operations. They are made of high-quality materials and meet industry safety standards. If you would like more information about the High-Visibility Vests or any other products, please let me know!'

In [27]:
memory.buffer

[HumanMessage(content='hi', additional_kwargs={}, example=False),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='what do you offer', additional_kwargs={}, example=False),
 AIMessage(content='At MineSafe Solutions, we offer a wide range of mining safety products to ensure the highest level of safety and protection in the mining industry. Some of our offerings include:\n\n1. Personal Protective Equipment (PPE): We provide a variety of PPE such as safety helmets, protective eyewear, ear protection, respiratory masks, high-visibility clothing, and safety gloves.\n\n2. Safety Devices: We offer safety devices like gas detectors, fire extinguishers, safety harnesses, fall protection systems, and emergency evacuation equipment.\n\n3. Advanced Monitoring Systems: Our advanced monitoring systems include real-time tracking and monitoring of personnel, equipment, and environmental conditions. This helps in ensuring the safety o

In [31]:
memory2.buffer

[]